In [68]:
from path_reader import PropagationPath,PathReader
import numpy as np
import torch
from PathNet import PathNet
import torch.optim as optim
from losses import FeatureMSE
from patch_utils import PatchGenerator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
paths = PathReader.read()

In [41]:
propogation_paths = []
for transmitter in paths:
    for receiver in transmitter:
        if len(receiver) != 0:
            propogation_paths.append(receiver)
propogation_paths = [item for sublist in propogation_paths for item in sublist]

In [42]:
path_vectors = [[path.path_gain_db, path.points, path.interaction_types] for path in propogation_paths]

In [43]:
# paths have 21 dimensions: 1 for path gain, 15 for points (5x3), 5 for interaction types
path_matrix = np.zeros((len(path_vectors),21))
for i, path in enumerate(path_vectors):
    path_matrix[i,0] = path[0]
    points = np.array(path[1]).flatten()
    path_matrix[i,1:1+len(points)] = points
    path_matrix[i,16:16+len(path[2])] = path[2]

In [44]:
path_net = PathNet(ic=21,outc=5)
optimizer = optim.Adam(path_net.parameters(), lr=0.001)
loss_func = FeatureMSE()

FeatureMSE locality: Non-local


In [45]:
for epoch in range(100):
    for i, path in enumerate(path_matrix):
        path = torch.from_numpy(path).float()
        optimizer.zero_grad()
        output = path_net(path)
        loss = loss_func(output, path)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print('Epoch: {} | Loss: {}'.format(epoch, loss.item()))

IndexError: too many indices for tensor of dimension 1

In [83]:
patch_generator = PatchGenerator(num_patches=8, attribute="transmitter")
np.random.shuffle(propogation_paths)
patches = patch_generator.generate_patches(propogation_paths[:50000])
print(patches)


C:\Users\thein\anaconda3\envs\pytorch\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[3 3 7 ... 6 1 6]
